<a href="https://colab.research.google.com/github/shumshersubashgautam/IR-searchengine/blob/main/information_retrieval_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

import nltk #NLTK for natural language processing tasks
from nltk.corpus import stopwords # list of stop word
from nltk.tokenize import word_tokenize # To tokenize each word
from nltk.stem import PorterStemmer # For specific rules to transform words to their stems
import ujson

# Preprosessing data before indexing
with open('result.json', 'r') as doc: scraper_results=doc.read()

# Initialize empty lists to store publication name, URL, author, and date

pubName = []
pubURL = []
pubCUAuthor = []
pubDate = []

# Load the scraped results using ujson
data_dict = ujson.loads(scraper_results)

# Get the length of the data_dict (number of publications)
array_length = len(data_dict)
# Print the number of publications
print(array_length)

#Seperate name, url, date, author in different file
for item in data_dict:
    pubName.append(item["name"])
    pubURL.append(item["pub_url"])
    pubCUAuthor.append(item["cu_author"])
    pubDate.append(item["date"])
with open('pub_name.json', 'w') as f:ujson.dump(pubName, f)
with open('pub_url.json', 'w') as f:ujson.dump(pubURL, f)
with open('pub_cu_author.json', 'w') as f:ujson.dump(pubCUAuthor, f)
with open('pub_date.json', 'w') as f: ujson.dump(pubDate, f)


#Open a file with publication names in read mode
with open('pub_name.json','r') as f:publication=f.read()

#Load JSON File
pubName = ujson.loads(publication)

#Downloading libraries to use its methods
nltk.download('stopwords')
nltk.download('punkt')

#Predefined stopwords in nltk are used
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
pub_list_first_stem = []
pub_list = []
pub_list_wo_sc = []
print(len(pubName))

for file in pubName:
    #Splitting strings to tokens(words)
    words = word_tokenize(file)
    stem_word = ""
    for i in words:
        if i.lower() not in stop_words:
            stem_word += stemmer.stem(i) + " "
    pub_list_first_stem.append(stem_word)
    pub_list.append(file)

#Removing all below characters
special_characters = '''!()-—[]{};:'"\, <>./?@#$%^&*_~0123456789+=’‘'''
for file in pub_list:
    word_wo_sc = ""
    if len(file.split()) ==1 : pub_list_wo_sc.append(file)
    else:
        for a in file:
            if a in special_characters:
                word_wo_sc += ' '
            else:
                word_wo_sc += a
        #print(word_wo_sc)
        pub_list_wo_sc.append(word_wo_sc)

#Stemming Process
pub_list_stem_wo_sw = []
for name in pub_list_wo_sc:
    words = word_tokenize(name)
    stem_word = ""
    for a in words:
        if a.lower() not in stop_words:
            stem_word += stemmer.stem(a) + ' '
    pub_list_stem_wo_sw.append(stem_word.lower())

data_dict = {} #Inverted Index holder

# Indexing process
for a in range(len(pub_list_stem_wo_sw)):
    for b in pub_list_stem_wo_sw[a].split():
        if b not in data_dict:
             data_dict[b] = [a]
        else:
            data_dict[b].append(a)

# printing the lenght
print(len(pub_list_wo_sc))
print(len(pub_list_stem_wo_sw))
print(len(pub_list_first_stem))
print(len(pub_list))

# with open('publication_list.json', 'w') as f:
#     ujson.dump(pub_list, f)

with open('publication_list_stemmed.json', 'w') as f:
    ujson.dump(pub_list_first_stem, f)

with open('publication_indexed_dictionary.json', 'w') as f:
    ujson.dump(data_dict, f)


696


KeyError: ignored

In [2]:
!pip install ujson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 5.1 MB/s eta 0:00:00


In [5]:
!git clone https://github.com/franklinobasy/Web-Scrape-Research-Publication.git

Cloning into 'Web-Scrape-Research-Publication'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 40 (delta 14), reused 38 (delta 12), pack-reused 0
Unpacking objects: 100% (40/40), 53.21 KiB | 2.96 MiB/s, done.


In [6]:
cd /content/Web-Scrape-Research-Publication/publication/

/content/Web-Scrape-Research-Publication/publication


In [15]:
from bs4 import BeautifulSoup
import json
import requests
from typing import List


class Extract():
    '''
    This class contains tools for webscraping Coventory University website
    for School of Economics, Finance and Accounting Research

    '''
    __url: str = "https://pureportal.coventry.ac.uk/en/organisations/centre-for-intelligent-healthcare/publications/"
    __pubs: List[dict] = []
    __headers: dict = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"
    }
    _page_to_parse: int = 14
    __has_started: bool = False

    def __init__(self, page_to_parse: int = 14):
        '''
        Argument:
            page_to_page: Numnber of pages to parse
        '''
        self._page_to_parse = page_to_parse

    @property
    def page_to_parse(self):
        return self._page_to_parse

    @page_to_parse.setter
    def page_to_parse(self, page_to_parse):
        if not isinstance(page_to_parse, int):
            raise TypeError(f"argument 'page_to_parse' must be an integer, \
                            {type(page_to_parse)} given")

        self._page_to_parse = page_to_parse

    @property
    def pubs(self):
        if self.__has_started:
            return self.__pubs
        else:
            raise TypeError("Web scrape engine has not been executed.")

    @classmethod
    def check_author_in_author_links(cls, authors_with_link: List[str], author: str) -> bool:
        '''
        Checks if an author is represent in list of authors with links

        Arguments:
            authors_with_links: a list of authors with links
            author: Name of author to check

        Return:
            Bool
        '''

        for author_ in authors_with_link:
            if author == author_:
                return True
        return False

    def webscrape(self) -> None:
        '''
        webscrape engine
        '''
        for page in range(self._page_to_parse):
            if page != 0:
                self.__url = f"https://pureportal.coventry.ac.uk/en/organisations/centre-for-intelligent-healthcare/publications/?page={page}"

            r = requests.get(self.__url, headers=self.__headers,)
            soup = BeautifulSoup(r.content, features="lxml")

            for item in soup.find_all("li", class_="list-result-item"):
                pub_object = {
                    "publication": "",
                    "publication_link": "",
                }

                if item.find_all("a", rel="ContributionToJournal"):
                    items = item.find_all("a", rel="ContributionToJournal")
                elif item.find_all("a", rel="ContributionToBookAnthology"):
                    items = item.find_all("a", rel="ContributionToBookAnthology")

                for pub in items:
                    pub_object["publication_link"] = pub['href']
                    pub_object['publication'] = pub.find("span").text

                # get authors with link
                authors_with_link = []
                char = 'a'

                for pub in item.find_all("a", class_="link person"):
                    link = pub.get("href", None)
                    author = pub.find("span").text
                    pub_object[f"author_{char}"] = author
                    pub_object[f"author_{char}_profile"] = link

                    authors_with_link.append(f"{author}")

                    char = chr(ord(char) + 1)

                # get authors without link
                for pub in item.find_all("span", class_="")[1:-1]:
                    if not Extract.check_author_in_author_links(authors_with_link, pub.text):
                        pub_object[f"author_{char}"] = pub.text
                        pub_object[f"author_{char}_profile"] = None
                        char = chr(ord(char) + 1)

                self.__pubs.append(pub_object)

    def run(self) -> None:
        '''
        Start Web scraping
        '''
        self.webscrape()
        self.__has_started = True

    def result(self) -> None:
        if not self.__has_started:
            raise TypeError("Call start method before calling result method")

        return self.__pubs

    def result_tojson(self, path: str) -> None:
        '''
        Export results to json

        Argument:
            path: export path

        Return:
            None
        '''
        results = self.result()

        if path.split('.')[-1] != "json":
            raise ValueError('File type must be a json')

        with open(path, "w") as f:
            json.dump(results, f)


if __name__ == "__main__":
    # instantiate extraction object
    extraction = Extract()

    # run the extraction
    extraction.run()

    # export result
    extraction.result_tojson('/content/scrapped_results.json')

In [18]:

import nltk #NLTK for natural language processing tasks
from nltk.corpus import stopwords # list of stop word
from nltk.tokenize import word_tokenize # To tokenize each word
from nltk.stem import PorterStemmer # For specific rules to transform words to their stems
import ujson

# Preprosessing data before indexing
with open('/content/scrapped_results.json', 'r') as doc: scraper_results=doc.read()

# Initialize empty lists to store publication name, URL, author, and date

pubName = []
pubURL = []
pubCUAuthor = []
pubauthprof = []

# Load the scraped results using ujson
data_dict = ujson.loads(scraper_results)

# Get the length of the data_dict (number of publications)
array_length = len(data_dict)
# Print the number of publications
print(array_length)

#Seperate name, url, date, author in different file
for item in data_dict:
    pubName.append(item["publication"])
    pubURL.append(item["publication_link"])
    pubCUAuthor.append(item.get("author_a", None))
    pubauthprof.append(item.get("author_a_profile", None))
with open('pub_name.json', 'w') as f:ujson.dump(pubName, f)
with open('pub_url.json', 'w') as f:ujson.dump(pubURL, f)
with open('pub_cu_author.json', 'w') as f:ujson.dump(pubCUAuthor, f)
with open('pub_date.json', 'w') as f: ujson.dump(pubauthprof, f)


#Open a file with publication names in read mode
with open('pub_name.json','r') as f:publication=f.read()

#Load JSON File
pubName = ujson.loads(publication)

#Downloading libraries to use its methods
nltk.download('stopwords')
nltk.download('punkt')

#Predefined stopwords in nltk are used
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
pub_list_first_stem = []
pub_list = []
pub_list_wo_sc = []
print(len(pubName))

for file in pubName:
    #Splitting strings to tokens(words)
    words = word_tokenize(file)
    stem_word = ""
    for i in words:
        if i.lower() not in stop_words:
            stem_word += stemmer.stem(i) + " "
    pub_list_first_stem.append(stem_word)
    pub_list.append(file)

#Removing all below characters
special_characters = '''!()-—[]{};:'"\, <>./?@#$%^&*_~0123456789+=’‘'''
for file in pub_list:
    word_wo_sc = ""
    if len(file.split()) ==1 : pub_list_wo_sc.append(file)
    else:
        for a in file:
            if a in special_characters:
                word_wo_sc += ' '
            else:
                word_wo_sc += a
        #print(word_wo_sc)
        pub_list_wo_sc.append(word_wo_sc)

#Stemming Process
pub_list_stem_wo_sw = []
for name in pub_list_wo_sc:
    words = word_tokenize(name)
    stem_word = ""
    for a in words:
        if a.lower() not in stop_words:
            stem_word += stemmer.stem(a) + ' '
    pub_list_stem_wo_sw.append(stem_word.lower())

data_dict = {} #Inverted Index holder

# Indexing process
for a in range(len(pub_list_stem_wo_sw)):
    for b in pub_list_stem_wo_sw[a].split():
        if b not in data_dict:
             data_dict[b] = [a]
        else:
            data_dict[b].append(a)

# printing the lenght
print(len(pub_list_wo_sc))
print(len(pub_list_stem_wo_sw))
print(len(pub_list_first_stem))
print(len(pub_list))

# with open('publication_list.json', 'w') as f:
#     ujson.dump(pub_list, f)

with open('publication_list_stemmed.json', 'w') as f:
    ujson.dump(pub_list_first_stem, f)

with open('publication_indexed_dictionary.json', 'w') as f:
    ujson.dump(data_dict, f)


696


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


696
696
696
696
696


In [29]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import ujson

# Load the JSON file containing scraped results
with open('/content/scrapped_results.json', 'r') as doc:
    scraper_results = doc.read()

# Extract author names from the JSON data
authors = []
data_dict = ujson.loads(scraper_results)
for item in data_dict:
    authors.append(item["author_a",  "author_b"])

# Write the author names to a JSON file
with open('author_names.json', 'w') as f:
    ujson.dump(authors, f)

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Load the JSON file containing author names
with open('author_names.json', 'r') as f:
    author_data = f.read()

# Load JSON data
authors = ujson.loads(author_data)

# Preprocess the author names
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
authors_list_first_stem = []
authors_list = []

for author in authors:
    words = word_tokenize(author)
    stem_word = ""
    for word in words:
        if word.lower() not in stop_words:
            stem_word += stemmer.stem(word) + " "
    authors_list_first_stem.append(stem_word)
    authors_list.append(author)

# Indexing process
data_dict = {}
for i in range(len(authors_list_first_stem)):
    for word in authors_list_first_stem[i].split():
        if word not in data_dict:
            data_dict[word] = [i]
        else:
            data_dict[word].append(i)

# Write the preprocessed author names and indexed dictionary to JSON files
with open('author_list_stemmed.json', 'w') as f:
    ujson.dump(authors_list_first_stem, f)

with open('author_indexed_dictionary.json', 'w') as f:
    ujson.dump(data_dict, f)

KeyError: ignored

In [24]:
print(item.keys())

dict_keys(['publication', 'publication_link', 'author_a', 'author_a_profile', 'author_b', 'author_b_profile', 'author_c', 'author_c_profile', 'author_d', 'author_d_profile', 'author_e', 'author_e_profile', 'author_f', 'author_f_profile', 'author_g', 'author_g_profile', 'author_h', 'author_h_profile'])


In [34]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import ujson

# Load the JSON file containing scraped results
with open('/content/scrapped_results.json', 'r') as doc:
    scraper_results = doc.read()

# Extract author names from the JSON data
authors = []
data_dict = ujson.loads(scraper_results)
for item in data_dict:
    if "author_a" in item:
        authors.append(item["author_a"])
    elif "author_b" in item:
        authors.append(item["author_b"])
    elif "author_c" in item:
        authors.append(item["author_c"])
    elif "author_d" in item:
        authors.append(item["author_d"])
    elif "author_e" in item:
        authors.append(item["author_e"])
    elif "author_f" in item:
        authors.append(item["author_f"])
    elif "author_g" in item:
        authors.append(item["author_g"])
    elif "author_h" in item:
        authors.append(item["author_h"])

# Write the author names to a JSON file
with open('author_names.json', 'w') as f:
    ujson.dump(authors, f)

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Load the JSON file containing author names
with open('author_names.json', 'r') as f:
    author_data = f.read()

# Load JSON data
authors = ujson.loads(author_data)

# Preprocess the author names
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
authors_list_first_stem = []
authors_list = []

for author in authors:
    words = word_tokenize(author)
    stem_word = ""
    for word in words:
        if word.lower() not in stop_words:
            stem_word += stemmer.stem(word) + " "
    stripped_stem_word = stem_word.strip() if stem_word else stem_word
    authors_list_first_stem.append(stripped_stem_word)
    authors_list.append(author)

# Indexing process
data_dict = {}
for i, stemmed_author in enumerate(authors_list_first_stem):
    for word in stemmed_author.split():
        if word not in data_dict:
            data_dict[word] = [i]
        else:
            data_dict[word].append(i)

# Write the preprocessed author names and indexed dictionary to JSON files
with open('author_list_stemmed.json', 'w') as f:
    ujson.dump(authors_list_first_stem, f)

with open('author_indexed_dictionary.json', 'w') as f:
    ujson.dump(data_dict, f)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import ujson

# Load the JSON file containing scraped results
with open('/content/scrapped_results.json', 'r') as doc:
    scraper_results = doc.read()

# Extract author names from the JSON data
authors = []
data_dict = ujson.loads(scraper_results)
for item in data_dict:
    for i in range(8):
        if f"author_{chr(ord('a') + i)}" in item:
            authors.append(item[f"author_{chr(ord('a') + i)}"])
            if f"author_{chr(ord('a') + i)}_profile" in item:
                authors.append(item[f"author_{chr(ord('a') + i)}_profile"])

# Write the author names to a JSON file
with open('author_names.json', 'w') as f:
    ujson.dump(authors, f)

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Load the JSON file containing author names
with open('author_names.json', 'r') as f:
    author_data = f.read()

# Load JSON data
authors = ujson.loads(author_data)

# Preprocess the author names
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
authors_list_first_stem = []
authors_list = []

for author in authors:
    words = word_tokenize(author)
    stem_word = ""
    for word in words:
        if word.lower() not in stop_words:
            stem_word += stemmer.stem(word) + " "
    authors_list_first_stem.append(stem_word.strip())
    authors_list.append(author)

# Indexing process
data_dict = {}
for i, stemmed_author in enumerate(authors_list_first_stem):
    for word in stemmed_author.split():
        if word not in data_dict:
            data_dict[word] = [i]
        else:
            data_dict[word].append(i)

# Write the preprocessed author names and indexed dictionary to JSON files
with open('author_list_stemmed.json', 'w') as f:
    ujson.dump(authors_list_first_stem, f)

with open('author_indexed_dictionary.json', 'w') as f:
    ujson.dump(data_dict, f)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


TypeError: ignored

In [33]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import ujson

# Load the JSON file containing scraped results
with open('/content/scrapped_results.json', 'r') as doc:
    scraper_results = doc.read()

# Extract author names from the JSON data
authors = []
data_dict = ujson.loads(scraper_results)
for item in data_dict:
    for i in range(8):
        if f"author_{chr(ord('a') + i)}" in item:
            authors.append(item[f"author_{chr(ord('a') + i)}"])
            if f"author_{chr(ord('a') + i)}_profile" in item:
                authors.append(item[f"author_{chr(ord('a') + i)}_profile"])

# Write the author names to a JSON file
with open('author_names.json', 'w') as f:
    ujson.dump(authors, f)

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Load the JSON file containing author names
with open('author_names.json', 'r') as f:
    author_data = f.read()

# Load JSON data
authors = ujson.loads(author_data)

# Preprocess the author names
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
authors_list_first_stem = []
authors_list = []

for author in authors:
    words = word_tokenize(author)
    stem_word = ""
    for word in words:
        if word.lower() not in stop_words:
            stem_word += stemmer.stem(word) + " "
    stripped_stem_word = stem_word.strip() if stem_word else stem_word
    authors_list_first_stem.append(stripped_stem_word)
    authors_list.append(author)

# Indexing process
data_dict = {}
for i, stemmed_author in enumerate(authors_list_first_stem):
    for word in stemmed_author.split():
        if word not in data_dict:
            data_dict[word] = [i]
        else:
            data_dict[word].append(i)

# Write the preprocessed author names and indexed dictionary to JSON files
with open('author_list_stemmed.json', 'w') as f:
    ujson.dump(authors_list_first_stem, f)

with open('author_indexed_dictionary.json', 'w') as f:
    ujson.dump(data_dict, f)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


TypeError: ignored

In [ ]:
def show_results(output_data, search_type):
    aa = 0
    rank_sorting = sorted(output_data.items(), key=lambda z: z[1], reverse=True)

    # Show the total number of research results
    st.info(f"Showing results for: {len(rank_sorting)}")

    # Show the cards
    N_cards_per_row = 3
    for n_row, (id_val, ranking) in enumerate(rank_sorting):
        i = n_row % N_cards_per_row
        if i == 0:
            st.write("---")
            cols = st.columns(N_cards_per_row, gap="large")
        # Draw the card
        with cols[n_row % N_cards_per_row]:
            if search_type == "Publications":
                pub_date_val = pub_date[id_val].strip() if pub_date[id_val] is not None else ""
                st.caption(pub_date_val)
                st.markdown(f"**{pub_cu_author[id_val].strip()}**")
                st.markdown(f"*{pub_name[id_val].strip()}*")
                st.markdown(f"**{pub_url[id_val]}**")
            elif search_type == "Authors":
                pub_date_val = pub_date[id_val].strip() if pub_date[id_val] is not None else ""
                st.caption(pub_date_val)
                st.markdown(f"**{author_name[id_val].strip()}**")
                st.markdown(f"*{pub_name[id_val].strip()}*")
                st.markdown(f"**{pub_url[id_val]}**")
                st.markdown(f"Ranking: {ranking[0]:.2f}")

        aa += 1

    if aa == 0:
        st.info("No results found. Please try again.")
    else:
        st.info(f"Results shown for: {aa}")
